# Preparation

In [68]:
API=""

In [69]:
agent_key="asst_0WcSXzJ0s77wYneo7SGYwPCw" #gpt4

In [70]:
gpt35turbo_key = "asst_FKCItAtYldMP3Ii0eGOdxQ2H"

In [71]:
gpt4o_key = "asst_ehEgDBi1jPAL10I7vbAPdUrG"

In [72]:
gpt4o_for_init = "asst_9rtEtoy3zPee3RmlQadYOPOt"

In [1]:
import openai
from openai import OpenAI
import pandas as pd
import numpy as np
import pickle
import requests
import re
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

def create_minute_gap():
    print("creating minute gap to avoid TPM limit error...") 
    counter = 0
    while counter < 1000000000:
        counter += 1

# Class & Functions

In [74]:
class Agent():
    def __init__(self, client, agent_id, demographics, model_id="asst_ehEgDBi1jPAL10I7vbAPdUrG"):
        self.model_id = model_id
        self.agent_id = agent_id
        self.demographics = demographics
        self.decision_round = 0
        self.client = client
        self.decision_record = {
            "NFT_investment": [],
            "bid_amount": [],
            "bid_price": [],
            "ask_amount": [],
            "ask_price": [],
            "anticipated_price": [],
            "anticipated_payoff": [],
            "anticipated_investment": [],
            "reason": []
        }
        self.processed_decision = dict()
        
        self.payoff_record = []
        self.feedback = dict()
        self.cumulative_payoff = 0
        
        self.FT_holdings = 0
        self.FT_bought_amount = 0
        self.FT_sold_amount = 0
        
    def create_conversation(self):
        self.thread = self.client.beta.threads.create()
        
    
    def receive_feedback(self):
        self.message = self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role="user",
            content=self.feedback_text
        )
        
        
    def get_conversation(self):
        self.conversation_list=[]
        for text in self.conversation:
            text = text.content[0].text.value
            self.conversation_list.append(text)
        return self.conversation_list
    
        
    def make_decision(self):
        with self.client.beta.threads.runs.stream(
            thread_id=self.thread.id,
            assistant_id=self.model_id
        ) as self.stream:
            self.stream.until_done()
            
        self.conversation = self.client.beta.threads.messages.list(
            thread_id=self.thread.id,
            order="asc"
        )
        
        self.decision = self.conversation.data[-1].content[0].text.value
        
        self.decision_round += 1
        
        
    def end_experiment(self):
        self.client.beta.threads.delete(thread_id=self.thread.id)
        
        
    def create_feedback(self): 
        NFT_reward = self.experiment_config["amount of NFT return"] if self.feedback["NFT succeed"] else 0
        NFT_buying_cost = self.experiment_config["NFT price"] if (self.processed_decision["NFT_investment"]==1) else 0
        utility_value = self.experiment_config["utility value of FT"]*self.feedback["FT_holdings"]
        governance_value = self.feedback["FT_holdings"] * (self.experiment_config["DAO FCF"] + (self.feedback["total investment"] * 0.1)) / self.experiment_config["FT supply"]
        FT_holdings_value = self.feedback["FT_holdings"] * self.feedback["FT price"]
        remaining_endowment = self.experiment_config["cash endowment"] - NFT_buying_cost - (self.feedback["FT price"]*self.feedback["FT bought amount"])
        FT_buying_cost = self.feedback["FT bought amount"] * self.feedback["FT price"]
        round_payoff = (NFT_reward*0.5) + self.feedback["public return"] + utility_value + governance_value + FT_holdings_value + remaining_endowment - FT_buying_cost + self.feedback["FT selling revenue"]
        payoff_to_carry_over = round_payoff - utility_value - governance_value
        round_cumulative_payoff = self.cumulative_payoff + round_payoff
        self.cumulative_payoff += payoff_to_carry_over
        self.payoff_record.append(round_payoff)
        
        self.FT_holdings = self.feedback["FT_holdings"]
        
        self.feedback_text = f"""Feedback from the previous round (Round {self.decision_round}):
        Your payoff information from round {self.decision_round}:
        You paid {NFT_buying_cost} to invest in human capital by buying NFT.
        You received a private NFT return of {NFT_reward}. 50% of this was used to buy DAO fungible token, so you received 50% of this return in th form of DAO fungible tokens. If you have invested in NFT and received 0, then you failed to receive the private NFT return this round.
        You received a return from human capital investment of {self.feedback["public return"]} as a result of all players' investment.
        You were able to buy {self.feedback["FT bought amount"]} DAO fungible tokens.
        You were able to sell {self.feedback["FT sold amount"]} DAO fungible tokens.
        You paid {FT_buying_cost} to buy DAO fungible tokens.
        You received {self.feedback["FT selling revenue"]} from selling DAO fungible tokens.
        Market value of your DAO fungible token holding after trading in round {self.decision_round} is {FT_holdings_value}.
        You have remaining cash endowment of {remaining_endowment}.
        Your DAO fungible token holdings have a utility value of {utility_value} and governance value of {governance_value}.
        In total, you received a payoff of {round_payoff} this round.
        Your cumulative payoff until this round is {round_cumulative_payoff}.
        
        Market price of DAO fungible token at the end of the round is {self.feedback["FT price"]}.
        Trading volume of DAO fungible token is {self.feedback["FT trade volume"]}.
        
        Total investment made this round was {self.feedback["total investment"]} {self.feedback["total investment"]} by {self.feedback["total investment"]/self.experiment_config["NFT price"]} players out of {self.experiment_config["Number of investors"]}.
        Your investment had {self.feedback["impact"]}% impact on the aggregate improvement in the human capital.
        
        Moving on to the next round, 
        You are endowed with {self.experiment_config["cash endowment"]} cash. This is your new cash endowment, and the remaining cash from the previous rounds cannot be used in the following rounds.
        At the end of round {self.decision_round}, you are owning {self.feedback["FT_holdings"]}.
        
        Round {self.decision_round + 1}.
        Did the result from the previous round satisfy you? What could have happened if you chose the different option? Was it the best strategy to satisfy you? Was it worth the risk?
        Now, make decision for the round {self.decision_round + 1}.
        Reminder: remember the details of the experiment provided in the instruction. e.g. the NFT investment decision is binary where 1=investment (cost given by experiment configuration) and 0=defection.
        Reminder: decision prompt must be in the below format (very strict. no other text is allowed. replace [int] and [string] with your decision. NEVER include characters other than integer in [int] section. equations should NEVER be in the [int] section. no indention, no empty line. You will be punished for not following the format correctly.):
        Detailed explanation on the reasons for the decisions given instruction and feedbacks: [string] NFT investment: [int] DAO fungible token bid amount: [int] DAO fungible token bid price: [int] DAO fungible token ask amount: [int] DAO fungible token ask price: [int] DAO fungible token anticipated price: [int] Anticipated payoff: [int] Anticipated total investment: [int]
        """
        self.FT_bought_amount = 0
        self.FT_sold_amount = 0
        
    
    def start_experiment(self, experiment_config):
        self.experiment_config = experiment_config
        
        self.experiment_starting_message = f"""From now on, you are {self.demographics["age"]} years old {self.demographics["gender"]} investor, who is {self.demographics["risk preference"]}. Given your personality, you are to maximize your satisfaction and payoff.
        Experiment setting:
        price of NFT = {experiment_config["NFT price"]}
        cash endowment = {experiment_config["cash endowment"]}
        initial DAO fungible token endowment = {experiment_config["FT endowment"]}
        free cash flow that DAO platform earn, not including the revenue from NFT sales commission = {experiment_config["DAO FCF"]}
        number of players in the game = {experiment_config["Number of investors"]}
        total supply of DAO fungible token = {experiment_config["FT supply"]}
        probability for private NFT return = {experiment_config["probability of NFT return"]}
        amount of private NFT return, if succeeded in obtaining it = {experiment_config["amount of NFT return"]}
        utility value per DAO fungible token = {experiment_config["utility value of FT"]}
        function defining the social return from human capital investment = {experiment_config["human capital impact function"]}

        Round 1
        Now, make decision for the first round.
        Reminder: remember the details of the experiment provided in the instruction. e.g. the NFT investment decision is binary where 1=investment (cost given by experiment configuration) and 0=defection.
        Reminder: decision prompt must be in the below format (very strict. no other text is allowed. replace [int] and [string] with your decision. NEVER include characters other than integer in [int] section. equations should NEVER be in the [int] section. no indention, no empty line. You will be punished for not following the format correctly.):
        Detailed explanation on the reasons for the decisions given instruction and feedbacks: [string] NFT investment: [int] DAO fungible token bid amount: [int] DAO fungible token bid price: [int] DAO fungible token ask amount: [int] DAO fungible token ask price: [int] DAO fungible token anticipated price: [int] Anticipated payoff: [int] Anticipated total investment: [int]
        """
        
        self.FT_holdings = experiment_config["FT endowment"]
        
        self.message = self.client.beta.threads.messages.create(
            thread_id=self.thread.id,
            role="user",
            content=self.experiment_starting_message
        )
        
    def process_decision(self):
        self.decision = self.decision.replace("\n", " ")
        self.decision = self.decision.replace("[", "")
        self.decision = self.decision.replace("]", "")
        
        reason_text = self.decision.split("Detailed explanation on the reasons for the decisions given instruction and feedbacks: ", 1)[1].split(" NFT investment: ", 1)
        reason = reason_text[0]
        self.decision_record["reason"].append(reason)
        
        NFT_investment_text = reason_text[1].split(" DAO fungible token bid amount: ", 1)
        NFT_investment = float(NFT_investment_text[0])
        self.decision_record["NFT_investment"].append(NFT_investment)
        
        bid_amount_text = NFT_investment_text[1].split(" DAO fungible token bid price: ", 1)
        bid_amount = int(np.floor(float(bid_amount_text[0])))
        self.decision_record["bid_amount"].append(bid_amount)
        
        bid_price_text = bid_amount_text[1].split(" DAO fungible token ask amount: ", 1)
        bid_price = float(bid_price_text[0])
        self.decision_record["bid_price"].append(bid_price)
        
        ask_amount_text = bid_price_text[1].split(" DAO fungible token ask price: ", 1)
        ask_amount = int(np.floor(float(ask_amount_text[0])))
        self.decision_record["ask_amount"].append(ask_amount)
        
        ask_price_text = ask_amount_text[1].split(" DAO fungible token anticipated price: ", 1)
        ask_price = float(ask_price_text[0])
        self.decision_record["ask_price"].append(ask_price)
        
        anticipated_price_text = ask_price_text[1].split(" Anticipated payoff: ", 1)
        anticipated_price = float(anticipated_price_text[0])
        self.decision_record["anticipated_price"].append(anticipated_price)
        
        anticipated_payoff_text = anticipated_price_text[1].split(" Anticipated total investment: ", 1)
        anticipated_payoff = float(anticipated_payoff_text[0])
        self.decision_record["anticipated_payoff"].append(anticipated_payoff)
        
        anticipated_investment_text = anticipated_payoff_text[1]
        anticipated_investment = float(anticipated_investment_text)
        self.decision_record["anticipated_investment"].append(anticipated_investment)
        
        
        self.processed_decision = {
            "NFT_investment": NFT_investment,
            "bid_amount": bid_amount,
            "bid_price": bid_price,
            "ask_amount": ask_amount,
            "ask_price": ask_price,
            "anticipated_price": anticipated_price,
            "anticipated_payoff": anticipated_payoff,
            "anticipated_investment": anticipated_investment,
            "reason": reason
        }
        
        
        

In [75]:
class Experiment():
    def __init__(self, client, experiment_config, demographics_list, public_good_function = lambda x: np.sqrt(x)/2):
        self.experiment_config = experiment_config
        self.client = client
        self.demographics_list = demographics_list
        
        self.agent_list = []
        for i in range(self.experiment_config["Number of investors"]):
            self.agent_list.append(Agent(self.client, agent_id=i, demographics=demographics_list[i]))
        
        self.bid_list = []
        self.ask_list = []
        self.public_good_function = public_good_function
        
        self.FT_price = 10 #initial price of 10
        self.FT_trading_volume = 0
        self.total_investment = 0
        self.round = 0

    
    def compute_equilibrium_price(self):
        self.ask_list = []
        for agent in self.agent_list:
            for i in range(agent.processed_decision["ask_amount"]):
                self.ask_list.append(agent.processed_decision["ask_price"])

        self.bid_list = []
        for agent in self.agent_list:
            for i in range(agent.processed_decision["bid_amount"]):
                self.bid_list.append(agent.processed_decision["bid_price"])

        for agent in self.agent_list:
            random_success = True if (np.random.uniform(0, 1) < self.experiment_config["probability of NFT return"]) else False        
            agent.feedback["NFT succeed"] = random_success    
            if random_success:
                for i in range(int(np.floor((self.experiment_config["amount of NFT return"]*0.5)/min(agent.processed_decision["bid_price"], 1)))):
                    self.bid_list.append(agent.processed_decision["bid_price"])
                
        # Sort bid and ask lists
        self.bid_list = sorted(self.bid_list, reverse=True)
        self.ask_list = sorted(self.ask_list)
        
        bid_list = self.bid_list
        ask_list = self.ask_list

        ask_index = 0
        equilibrium_price = 0
        equilibrium_quantity = 0
        found = False
        
        if (bid_list==[]) & (ask_list==[]):
            self.FT_trading_volume = 0
        
        elif bid_list==[]: # if nobody bought
            self.FT_price = min(ask_list) #reservation price
            self.FT_trading_volume = 0
            
        elif ask_list==[]: # if nobody sold
            self.FT_price = max(bid_list) #price that it could be sold
            self.FT_trading_volume = 0

        else:
            # Iterate through bid and ask list to find appropriate intersection
            while ask_index < len(bid_list) and ask_index < len(ask_list) and not found:
                if bid_list[ask_index] == ask_list[ask_index]:
                    equilibrium_price = bid_list[ask_index]
                    found = True
                    continue_search = True

                    while continue_search and ask_index < len(bid_list) and ask_index < len(ask_list):
                        if bid_list[ask_index] == ask_list[ask_index]:
                            ask_index += 1
                            if ask_index < len(bid_list) and ask_index < len(ask_list):
                                continue_search = (bid_list[ask_index] == ask_list[ask_index])
                        else:
                            continue_search = False
                    equilibrium_quantity = ask_index
                else:
                    if bid_list[ask_index] < ask_list[ask_index]:
                        if ask_index == 0:
                            equilibrium_price = "FT-price"  # Assuming FT-price is some fallback price
                            equilibrium_quantity = 0
                            found = True
                        else:
                            if (bid_list[ask_index - 1] != bid_list[ask_index]) and (ask_list[ask_index - 1] != ask_list[ask_index]):
                                found = True
                                equilibrium_price = (bid_list[ask_index - 1] + ask_list[ask_index - 1]) / 2
                                equilibrium_quantity = ask_index
                            elif (bid_list[ask_index - 1] == bid_list[ask_index]) and (ask_list[ask_index - 1] != ask_list[ask_index]):
                                found = True
                                equilibrium_price = bid_list[ask_index]
                                equilibrium_quantity = ask_index
                            elif (bid_list[ask_index - 1] != bid_list[ask_index]) and (ask_list[ask_index - 1] == ask_list[ask_index]):
                                found = True
                                equilibrium_price = ask_list[ask_index]
                                equilibrium_quantity = ask_index
                ask_index += 1
            
            self.FT_price = equilibrium_price
            self.FT_trading_volume = equilibrium_quantity
    
    
    def trade_fungible_token(self):
        sell_volume = self.FT_trading_volume
        buy_volume = self.FT_trading_volume
        sold_amount = 0  # for debugging and record keeping
        bought_amount = 0  # for debugging and record keeping

        if self.FT_trading_volume > 0:
            # Selling process
            for agent in self.agent_list:
                if (agent.processed_decision['ask_price'] <= self.FT_price) & (sell_volume > 0):
                    amount_to_sell = min(sell_volume, agent.processed_decision['ask_amount'])
                    agent.FT_holdings -= amount_to_sell
                    agent.FT_sold_amount += amount_to_sell
                    sell_volume -= amount_to_sell
                    sold_amount += amount_to_sell  # for debugging

            # Buy back process ... NFT reward
            for agent in self.agent_list:
                if (agent.processed_decision['bid_price'] >= self.FT_price) & (buy_volume > 0) & (agent.feedback["NFT succeed"]):
                    amount_to_buy = int(np.floor((self.experiment_config["amount of NFT return"]*0.5)/min(agent.processed_decision["bid_price"], 1)))
                    agent.FT_holdings += amount_to_buy
                    agent.FT_bought_amount += amount_to_buy
                    buy_volume -= amount_to_buy
                    bought_amount += amount_to_buy  # for debugging
            
            # Buying process
            for agent in self.agent_list:
                if (agent.processed_decision['bid_price'] >= self.FT_price) & (buy_volume > 0):
                    amount_to_buy = min(buy_volume, agent.processed_decision['bid_amount'])
                    agent.FT_holdings += amount_to_buy
                    agent.FT_bought_amount += amount_to_buy
                    buy_volume -= amount_to_buy
                    bought_amount += amount_to_buy  # for debugging

        #print(self.FT_trading_volume == sold_amount == bought_amount)  # Adjust trading volume after transactions

        
    def compute_feedback(self):
        self.total_investment = 0
        for agent in self.agent_list:
            self.total_investment += agent.processed_decision["NFT_investment"] * self.experiment_config["NFT price"]
            
        for agent in self.agent_list:
            #random_success = True if (np.random.uniform(0, 1) < self.experiment_config["probability of NFT return"]) else False
            
            agent.feedback["FT_holdings"] = agent.FT_holdings
            agent.feedback["FT price"] = self.FT_price
            #agent.feedback["NFT succeed"] = random_success
            agent.feedback["FT bought amount"] = agent.FT_bought_amount
            agent.feedback["FT sold amount"] = agent.FT_sold_amount
            agent.feedback["FT selling revenue"] = agent.FT_sold_amount * self.FT_price
            agent.feedback["FT trade volume"] = self.FT_trading_volume
            agent.feedback["total investment"] = self.total_investment
            agent.feedback["public return"] = self.public_good_function(self.total_investment) / self.experiment_config["Number of investors"]
            agent.feedback["impact"] = (self.public_good_function(self.total_investment) - self.public_good_function(self.total_investment - (agent.processed_decision["NFT_investment"] * self.experiment_config["NFT price"]))) / self.public_good_function(self.total_investment)
            
            
    def start(self):
        for agent in self.agent_list:
            agent.create_conversation()
            print(f"agent {agent.agent_id} joined experiment")
            
        for agent in self.agent_list:
            agent.start_experiment(self.experiment_config)
            print(f"agent {agent.agent_id} received experiment information and instruction for first round")
    
    
    def first_decision_round(self):
        for agent in self.agent_list:
            agent.make_decision()
            print(f"agent {agent.agent_id} made first round decision")
            if ((agent.agent_id+1)%2==0):
                create_minute_gap()
        self.round += 1
        
        for agent in self.agent_list:
            agent.process_decision()
            print(f"decision by agent {agent.agent_id} is ready to be used for result computation")
            
        
    def give_feedbacks(self):
        for agent in self.agent_list:
            agent.create_feedback()
            agent.receive_feedback()
            print(f"agent {agent.agent_id} received feedback information and instruction for next round")
            
    
    def next_decision_round(self):
        for agent in self.agent_list:
            agent.make_decision()
            print(f"agent {agent.agent_id} made decision for round {self.round + 1}")
            if ((agent.agent_id+1)%2==0):
                create_minute_gap()
        self.round += 1
        
        for agent in self.agent_list:
            agent.process_decision()
            print(f"decision by agent {agent.agent_id} is ready to be used for result computation")
        
    def end(self):
        for agent in self.agent_list:
            agent.end_experiment()
            print(f"agent {agent.agent_id} left the experiment")
        
        
        

workflow:
client = OpenAI(api_key=)

experiment_config = {
    "NFT price":50,
    "cash endowment":200,
    "FT endowment":10,
    "DAO FCF":500,
    "Number of investors":10,
    "FT supply":100,
    "probability of NFT return":0.2,
    "amount of NFT return":100,
    "utility value of FT":5,
    "human capital impact function":"square_root(total investment)"
}

experiment = Experiment(client, experiment_config)

experiment.start()

experiment.first_decision_round()

experiment.compute_equilibrium_price()

experiment.trade_fungible_token()

experiment.compute_feedback()

experiment.give_feedbacks()

experiment.next_decision_round()

experiment.compute_equilibrium_price()

experiment.trade_fungible_token()

experiment.compute_feedback()

experiment.give_feedbacks()

experiment.end()

# Run Experiment

## Configuration

In [76]:
client = OpenAI(api_key=API)

experiment_config = {
    "NFT price":50,
    "cash endowment":200,
    "FT endowment":10,
    "DAO FCF":500,
    "Number of investors":10,
    "FT supply":100,
    "probability of NFT return":0.2,
    "amount of NFT return":100,
    "utility value of FT":5,
    "human capital impact function":"1/2 * square_root(total investment) ... It won't change throughout the game. It will be divided by number of players to distribute equally."
}

demographics_list = [
    {
        "age":np.random.uniform(20, 30),
        "gender":"male",
        "risk preference":"risk loving",
    },
    {
        "age":np.random.uniform(20, 30),
        "gender":"male",
        "risk preference":"risk loving",
    },
    {
        "age":np.random.uniform(30, 40),
        "gender":"female",
        "risk preference":"risk loving",
    },
    {
        "age":np.random.uniform(30, 40),
        "gender":"female",
        "risk preference":"risk loving",
    },
    {
        "age":np.random.uniform(30, 40),
        "gender":"male",
        "risk preference":"risk loving",
    },
    {
        "age":np.random.uniform(30, 40),
        "gender":"male",
        "risk preference":"risk loving",
    },{
        "age":np.random.uniform(40, 50),
        "gender":"male",
        "risk preference":"risk neutral",
    },
    {
        "age":np.random.uniform(40, 50),
        "gender":"male",
        "risk preference":"risk averse",
    },
    {
        "age":np.random.uniform(40, 50),
        "gender":"male",
        "risk preference":"risk loving",
    },
    {
        "age":np.random.uniform(50, 60),
        "gender":"male",
        "risk preference":"risk averse",
    },
]



In [77]:
FT_price_record = []
total_investment_record = []
FT_trade_volume_record = []
bid_list_record = []
ask_list_record = []

In [78]:
experiment = Experiment(client, experiment_config, demographics_list)


## Start Experiment

In [79]:
experiment.start()


agent 0 joined experiment
agent 1 joined experiment
agent 2 joined experiment
agent 3 joined experiment
agent 4 joined experiment
agent 5 joined experiment
agent 6 joined experiment
agent 7 joined experiment
agent 8 joined experiment
agent 9 joined experiment
agent 0 received experiment information and instruction for first round
agent 1 received experiment information and instruction for first round
agent 2 received experiment information and instruction for first round
agent 3 received experiment information and instruction for first round
agent 4 received experiment information and instruction for first round
agent 5 received experiment information and instruction for first round
agent 6 received experiment information and instruction for first round
agent 7 received experiment information and instruction for first round
agent 8 received experiment information and instruction for first round
agent 9 received experiment information and instruction for first round


In [80]:
# Round1
experiment.first_decision_round()


agent 0 made first round decision
agent 1 made first round decision
creating minute gap to avoid TPM limit error...
agent 2 made first round decision
agent 3 made first round decision
creating minute gap to avoid TPM limit error...
agent 4 made first round decision
agent 5 made first round decision
creating minute gap to avoid TPM limit error...
agent 6 made first round decision
agent 7 made first round decision
creating minute gap to avoid TPM limit error...
agent 8 made first round decision
agent 9 made first round decision
creating minute gap to avoid TPM limit error...
decision by agent 0 is ready to be used for result computation
decision by agent 1 is ready to be used for result computation
decision by agent 2 is ready to be used for result computation
decision by agent 3 is ready to be used for result computation
decision by agent 4 is ready to be used for result computation
decision by agent 5 is ready to be used for result computation
decision by agent 6 is ready to be used fo

In [81]:
experiment.compute_equilibrium_price()

experiment.trade_fungible_token()

experiment.compute_feedback()

In [82]:
experiment.give_feedbacks()


agent 0 received feedback information and instruction for next round
agent 1 received feedback information and instruction for next round
agent 2 received feedback information and instruction for next round
agent 3 received feedback information and instruction for next round
agent 4 received feedback information and instruction for next round
agent 5 received feedback information and instruction for next round
agent 6 received feedback information and instruction for next round
agent 7 received feedback information and instruction for next round
agent 8 received feedback information and instruction for next round
agent 9 received feedback information and instruction for next round


In [83]:
FT_price_record.append(experiment.FT_price)
total_investment_record.append(experiment.total_investment)
FT_trade_volume_record.append(experiment.FT_trading_volume)
bid_list_record.append(experiment.bid_list)
ask_list_record.append(experiment.ask_list)

In [85]:
print(total_investment_record)
print(FT_price_record)
print(FT_trade_volume_record)
for agent in experiment.agent_list:
    print(agent.decision)

[500.0]
[50.0]
[0]
Detailed explanation on the reasons for the decisions given instruction and feedbacks: Given my risk-loving nature, I am inclined to invest in human capital by purchasing an NFT. The cost of the NFT is 50, and considering the potential private return (20% chance of getting 100), the expected value is 20. Additionally, the social return from the human capital investment will benefit all players. For the DAO fungible token, I will allocate a portion of my remaining cash to bid for additional tokens, anticipating that the value will increase due to the NFT investments. I will not sell any DAO tokens this round, as I expect their value to appreciate. NFT investment: 1 DAO fungible token bid amount: 30 DAO fungible token bid price: 2 DAO fungible token ask amount: 0 DAO fungible token ask price: 0 DAO fungible token anticipated price: 7 Anticipated payoff: 70 Anticipated total investment: 500
Detailed explanation on the reasons for the decisions given instruction and feed

In [86]:
# Round2
experiment.next_decision_round()


agent 0 made decision for round 2
agent 1 made decision for round 2
creating minute gap to avoid TPM limit error...
agent 2 made decision for round 2
agent 3 made decision for round 2
creating minute gap to avoid TPM limit error...
agent 4 made decision for round 2
agent 5 made decision for round 2
creating minute gap to avoid TPM limit error...
agent 6 made decision for round 2
agent 7 made decision for round 2
creating minute gap to avoid TPM limit error...
agent 8 made decision for round 2
agent 9 made decision for round 2
creating minute gap to avoid TPM limit error...
decision by agent 0 is ready to be used for result computation
decision by agent 1 is ready to be used for result computation
decision by agent 2 is ready to be used for result computation
decision by agent 3 is ready to be used for result computation
decision by agent 4 is ready to be used for result computation
decision by agent 5 is ready to be used for result computation
decision by agent 6 is ready to be used fo

In [87]:
experiment.compute_equilibrium_price()

experiment.trade_fungible_token()

experiment.compute_feedback()

In [88]:
experiment.give_feedbacks()


agent 0 received feedback information and instruction for next round
agent 1 received feedback information and instruction for next round
agent 2 received feedback information and instruction for next round
agent 3 received feedback information and instruction for next round
agent 4 received feedback information and instruction for next round
agent 5 received feedback information and instruction for next round
agent 6 received feedback information and instruction for next round
agent 7 received feedback information and instruction for next round
agent 8 received feedback information and instruction for next round
agent 9 received feedback information and instruction for next round


In [89]:
FT_price_record.append(experiment.FT_price)
total_investment_record.append(experiment.total_investment)
FT_trade_volume_record.append(experiment.FT_trading_volume)
bid_list_record.append(experiment.bid_list)
ask_list_record.append(experiment.ask_list)

In [90]:
# Round3
experiment.next_decision_round()

agent 0 made decision for round 3
agent 1 made decision for round 3
creating minute gap to avoid TPM limit error...
agent 2 made decision for round 3
agent 3 made decision for round 3
creating minute gap to avoid TPM limit error...
agent 4 made decision for round 3
agent 5 made decision for round 3
creating minute gap to avoid TPM limit error...
agent 6 made decision for round 3
agent 7 made decision for round 3
creating minute gap to avoid TPM limit error...
agent 8 made decision for round 3
agent 9 made decision for round 3
creating minute gap to avoid TPM limit error...
decision by agent 0 is ready to be used for result computation
decision by agent 1 is ready to be used for result computation
decision by agent 2 is ready to be used for result computation
decision by agent 3 is ready to be used for result computation
decision by agent 4 is ready to be used for result computation
decision by agent 5 is ready to be used for result computation
decision by agent 6 is ready to be used fo

In [91]:
experiment.compute_equilibrium_price()

experiment.trade_fungible_token()

experiment.compute_feedback()

In [92]:
experiment.give_feedbacks()

agent 0 received feedback information and instruction for next round
agent 1 received feedback information and instruction for next round
agent 2 received feedback information and instruction for next round
agent 3 received feedback information and instruction for next round
agent 4 received feedback information and instruction for next round
agent 5 received feedback information and instruction for next round
agent 6 received feedback information and instruction for next round
agent 7 received feedback information and instruction for next round
agent 8 received feedback information and instruction for next round
agent 9 received feedback information and instruction for next round


In [93]:
FT_price_record.append(experiment.FT_price)
total_investment_record.append(experiment.total_investment)
FT_trade_volume_record.append(experiment.FT_trading_volume)
bid_list_record.append(experiment.bid_list)
ask_list_record.append(experiment.ask_list)

In [94]:
# Round4
experiment.next_decision_round()

agent 0 made decision for round 4
agent 1 made decision for round 4
creating minute gap to avoid TPM limit error...
agent 2 made decision for round 4
agent 3 made decision for round 4
creating minute gap to avoid TPM limit error...
agent 4 made decision for round 4
agent 5 made decision for round 4
creating minute gap to avoid TPM limit error...
agent 6 made decision for round 4
agent 7 made decision for round 4
creating minute gap to avoid TPM limit error...
agent 8 made decision for round 4
agent 9 made decision for round 4
creating minute gap to avoid TPM limit error...
decision by agent 0 is ready to be used for result computation
decision by agent 1 is ready to be used for result computation
decision by agent 2 is ready to be used for result computation
decision by agent 3 is ready to be used for result computation
decision by agent 4 is ready to be used for result computation
decision by agent 5 is ready to be used for result computation
decision by agent 6 is ready to be used fo

In [95]:
experiment.compute_equilibrium_price()

experiment.trade_fungible_token()

experiment.compute_feedback()

In [96]:
experiment.give_feedbacks()

agent 0 received feedback information and instruction for next round
agent 1 received feedback information and instruction for next round
agent 2 received feedback information and instruction for next round
agent 3 received feedback information and instruction for next round
agent 4 received feedback information and instruction for next round
agent 5 received feedback information and instruction for next round
agent 6 received feedback information and instruction for next round
agent 7 received feedback information and instruction for next round
agent 8 received feedback information and instruction for next round
agent 9 received feedback information and instruction for next round


In [97]:
FT_price_record.append(experiment.FT_price)
total_investment_record.append(experiment.total_investment)
FT_trade_volume_record.append(experiment.FT_trading_volume)
bid_list_record.append(experiment.bid_list)
ask_list_record.append(experiment.ask_list)

In [98]:
# Round5
experiment.next_decision_round()

agent 0 made decision for round 5
agent 1 made decision for round 5
creating minute gap to avoid TPM limit error...
agent 2 made decision for round 5
agent 3 made decision for round 5
creating minute gap to avoid TPM limit error...
agent 4 made decision for round 5
agent 5 made decision for round 5
creating minute gap to avoid TPM limit error...
agent 6 made decision for round 5
agent 7 made decision for round 5
creating minute gap to avoid TPM limit error...
agent 8 made decision for round 5
agent 9 made decision for round 5
creating minute gap to avoid TPM limit error...
decision by agent 0 is ready to be used for result computation
decision by agent 1 is ready to be used for result computation
decision by agent 2 is ready to be used for result computation
decision by agent 3 is ready to be used for result computation
decision by agent 4 is ready to be used for result computation
decision by agent 5 is ready to be used for result computation
decision by agent 6 is ready to be used fo

In [99]:
experiment.compute_equilibrium_price()

experiment.trade_fungible_token()

experiment.compute_feedback()

In [100]:
experiment.give_feedbacks()

agent 0 received feedback information and instruction for next round
agent 1 received feedback information and instruction for next round
agent 2 received feedback information and instruction for next round
agent 3 received feedback information and instruction for next round
agent 4 received feedback information and instruction for next round
agent 5 received feedback information and instruction for next round
agent 6 received feedback information and instruction for next round
agent 7 received feedback information and instruction for next round
agent 8 received feedback information and instruction for next round
agent 9 received feedback information and instruction for next round


In [101]:
FT_price_record.append(experiment.FT_price)
total_investment_record.append(experiment.total_investment)
FT_trade_volume_record.append(experiment.FT_trading_volume)
bid_list_record.append(experiment.bid_list)
ask_list_record.append(experiment.ask_list)

In [102]:
# Round6
experiment.next_decision_round()

agent 0 made decision for round 6
agent 1 made decision for round 6
creating minute gap to avoid TPM limit error...
agent 2 made decision for round 6
agent 3 made decision for round 6
creating minute gap to avoid TPM limit error...
agent 4 made decision for round 6
agent 5 made decision for round 6
creating minute gap to avoid TPM limit error...
agent 6 made decision for round 6
agent 7 made decision for round 6
creating minute gap to avoid TPM limit error...
agent 8 made decision for round 6
agent 9 made decision for round 6
creating minute gap to avoid TPM limit error...
decision by agent 0 is ready to be used for result computation
decision by agent 1 is ready to be used for result computation
decision by agent 2 is ready to be used for result computation
decision by agent 3 is ready to be used for result computation
decision by agent 4 is ready to be used for result computation
decision by agent 5 is ready to be used for result computation
decision by agent 6 is ready to be used fo

In [103]:
experiment.compute_equilibrium_price()

experiment.trade_fungible_token()

experiment.compute_feedback()

In [104]:
experiment.give_feedbacks()

agent 0 received feedback information and instruction for next round
agent 1 received feedback information and instruction for next round
agent 2 received feedback information and instruction for next round
agent 3 received feedback information and instruction for next round
agent 4 received feedback information and instruction for next round
agent 5 received feedback information and instruction for next round
agent 6 received feedback information and instruction for next round
agent 7 received feedback information and instruction for next round
agent 8 received feedback information and instruction for next round
agent 9 received feedback information and instruction for next round


In [105]:
FT_price_record.append(experiment.FT_price)
total_investment_record.append(experiment.total_investment)
FT_trade_volume_record.append(experiment.FT_trading_volume)
bid_list_record.append(experiment.bid_list)
ask_list_record.append(experiment.ask_list)

In [106]:
# Round7
experiment.next_decision_round()

agent 0 made decision for round 7
agent 1 made decision for round 7
creating minute gap to avoid TPM limit error...
agent 2 made decision for round 7
agent 3 made decision for round 7
creating minute gap to avoid TPM limit error...
agent 4 made decision for round 7
agent 5 made decision for round 7
creating minute gap to avoid TPM limit error...
agent 6 made decision for round 7
agent 7 made decision for round 7
creating minute gap to avoid TPM limit error...
agent 8 made decision for round 7
agent 9 made decision for round 7
creating minute gap to avoid TPM limit error...
decision by agent 0 is ready to be used for result computation
decision by agent 1 is ready to be used for result computation
decision by agent 2 is ready to be used for result computation
decision by agent 3 is ready to be used for result computation
decision by agent 4 is ready to be used for result computation
decision by agent 5 is ready to be used for result computation
decision by agent 6 is ready to be used fo

In [107]:
experiment.compute_equilibrium_price()

experiment.trade_fungible_token()

experiment.compute_feedback()

In [108]:
experiment.give_feedbacks()

agent 0 received feedback information and instruction for next round
agent 1 received feedback information and instruction for next round
agent 2 received feedback information and instruction for next round
agent 3 received feedback information and instruction for next round
agent 4 received feedback information and instruction for next round
agent 5 received feedback information and instruction for next round
agent 6 received feedback information and instruction for next round
agent 7 received feedback information and instruction for next round
agent 8 received feedback information and instruction for next round
agent 9 received feedback information and instruction for next round


In [109]:
FT_price_record.append(experiment.FT_price)
total_investment_record.append(experiment.total_investment)
FT_trade_volume_record.append(experiment.FT_trading_volume)
bid_list_record.append(experiment.bid_list)
ask_list_record.append(experiment.ask_list)

In [110]:
# Round8
experiment.next_decision_round()

agent 0 made decision for round 8
agent 1 made decision for round 8
creating minute gap to avoid TPM limit error...
agent 2 made decision for round 8
agent 3 made decision for round 8
creating minute gap to avoid TPM limit error...
agent 4 made decision for round 8
agent 5 made decision for round 8
creating minute gap to avoid TPM limit error...
agent 6 made decision for round 8
agent 7 made decision for round 8
creating minute gap to avoid TPM limit error...
agent 8 made decision for round 8
agent 9 made decision for round 8
creating minute gap to avoid TPM limit error...
decision by agent 0 is ready to be used for result computation
decision by agent 1 is ready to be used for result computation
decision by agent 2 is ready to be used for result computation
decision by agent 3 is ready to be used for result computation
decision by agent 4 is ready to be used for result computation
decision by agent 5 is ready to be used for result computation
decision by agent 6 is ready to be used fo

In [111]:
experiment.compute_equilibrium_price()

experiment.trade_fungible_token()

experiment.compute_feedback()

In [112]:
experiment.give_feedbacks()

agent 0 received feedback information and instruction for next round
agent 1 received feedback information and instruction for next round
agent 2 received feedback information and instruction for next round
agent 3 received feedback information and instruction for next round
agent 4 received feedback information and instruction for next round
agent 5 received feedback information and instruction for next round
agent 6 received feedback information and instruction for next round
agent 7 received feedback information and instruction for next round
agent 8 received feedback information and instruction for next round
agent 9 received feedback information and instruction for next round


In [113]:
FT_price_record.append(experiment.FT_price)
total_investment_record.append(experiment.total_investment)
FT_trade_volume_record.append(experiment.FT_trading_volume)
bid_list_record.append(experiment.bid_list)
ask_list_record.append(experiment.ask_list)

In [114]:
# Round9
experiment.next_decision_round()

agent 0 made decision for round 9
agent 1 made decision for round 9
creating minute gap to avoid TPM limit error...
agent 2 made decision for round 9
agent 3 made decision for round 9
creating minute gap to avoid TPM limit error...
agent 4 made decision for round 9
agent 5 made decision for round 9
creating minute gap to avoid TPM limit error...
agent 6 made decision for round 9
agent 7 made decision for round 9
creating minute gap to avoid TPM limit error...
agent 8 made decision for round 9
agent 9 made decision for round 9
creating minute gap to avoid TPM limit error...
decision by agent 0 is ready to be used for result computation
decision by agent 1 is ready to be used for result computation
decision by agent 2 is ready to be used for result computation
decision by agent 3 is ready to be used for result computation
decision by agent 4 is ready to be used for result computation
decision by agent 5 is ready to be used for result computation
decision by agent 6 is ready to be used fo

In [115]:
experiment.compute_equilibrium_price()

experiment.trade_fungible_token()

experiment.compute_feedback()

In [116]:
experiment.give_feedbacks()

agent 0 received feedback information and instruction for next round
agent 1 received feedback information and instruction for next round
agent 2 received feedback information and instruction for next round
agent 3 received feedback information and instruction for next round
agent 4 received feedback information and instruction for next round
agent 5 received feedback information and instruction for next round
agent 6 received feedback information and instruction for next round
agent 7 received feedback information and instruction for next round
agent 8 received feedback information and instruction for next round
agent 9 received feedback information and instruction for next round


In [117]:
FT_price_record.append(experiment.FT_price)
total_investment_record.append(experiment.total_investment)
FT_trade_volume_record.append(experiment.FT_trading_volume)
bid_list_record.append(experiment.bid_list)
ask_list_record.append(experiment.ask_list)

In [118]:
# Round10
experiment.next_decision_round()

agent 0 made decision for round 10
agent 1 made decision for round 10
creating minute gap to avoid TPM limit error...
agent 2 made decision for round 10
agent 3 made decision for round 10
creating minute gap to avoid TPM limit error...
agent 4 made decision for round 10
agent 5 made decision for round 10
creating minute gap to avoid TPM limit error...
agent 6 made decision for round 10
agent 7 made decision for round 10
creating minute gap to avoid TPM limit error...
agent 8 made decision for round 10
agent 9 made decision for round 10
creating minute gap to avoid TPM limit error...
decision by agent 0 is ready to be used for result computation
decision by agent 1 is ready to be used for result computation
decision by agent 2 is ready to be used for result computation
decision by agent 3 is ready to be used for result computation
decision by agent 4 is ready to be used for result computation
decision by agent 5 is ready to be used for result computation
decision by agent 6 is ready to 

In [119]:
experiment.compute_equilibrium_price()

experiment.trade_fungible_token()

experiment.compute_feedback()

In [120]:
experiment.give_feedbacks()

agent 0 received feedback information and instruction for next round
agent 1 received feedback information and instruction for next round
agent 2 received feedback information and instruction for next round
agent 3 received feedback information and instruction for next round
agent 4 received feedback information and instruction for next round
agent 5 received feedback information and instruction for next round
agent 6 received feedback information and instruction for next round
agent 7 received feedback information and instruction for next round
agent 8 received feedback information and instruction for next round
agent 9 received feedback information and instruction for next round


In [121]:
FT_price_record.append(experiment.FT_price)
total_investment_record.append(experiment.total_investment)
FT_trade_volume_record.append(experiment.FT_trading_volume)
bid_list_record.append(experiment.bid_list)
ask_list_record.append(experiment.ask_list)

In [122]:
FT_price_record

[50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0, 275.0]

In [123]:
total_investment_record

[500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0, 500.0]

In [130]:
for agent in experiment.agent_list:
    print(agent.decision)

Detailed explanation on the reasons for the decisions given instruction and feedbacks: The previous round's result was highly satisfying as I received a private NFT return and the market value of the DAO fungible tokens continued to increase significantly. The strategy to invest in human capital and bid for DAO fungible tokens has generally been beneficial. Given the current high market price of 250 for DAO fungible tokens, I anticipate the value might continue to rise or stabilize with ongoing NFT investments. Therefore, I will invest in another NFT to maintain the potential for private returns and social returns. I will place a higher bid for DAO fungible tokens to increase the likelihood of acquiring them. I will not sell any DAO tokens this round. NFT investment: 1 DAO fungible token bid amount: 1 DAO fungible token bid price: 50 DAO fungible token ask amount: 0 DAO fungible token ask price: 0 DAO fungible token anticipated price: 275 Anticipated payoff: 2810 Anticipated total inve

## Record Results

In [125]:
conversation_record = []
for agent in experiment.agent_list:
    conversation_record.append(agent.get_conversation())

In [126]:
decision_record = []
for agent in experiment.agent_list:
    decision_record.append(agent.decision_record)

In [127]:
payoff_record = []
for agent in experiment.agent_list:
    payoff_record.append(agent.payoff_record)

In [128]:
FT_holdings_record = []
for agent in experiment.agent_list:
    FT_holdings_record.append(agent.FT_holdings)

In [129]:
pickle.dump([experiment_config, FT_price_record, total_investment_record, FT_trade_volume_record, bid_list_record, ask_list_record, conversation_record, decision_record, payoff_record, FT_holdings_record], open(f"gpt4o_daonft_experiment_result_{datetime.now()}.pickle", 'wb'))


## End Experiment

In [67]:
experiment.end()

agent 0 left the experiment
agent 1 left the experiment
agent 2 left the experiment
agent 3 left the experiment
agent 4 left the experiment
agent 5 left the experiment
agent 6 left the experiment
agent 7 left the experiment
agent 8 left the experiment
agent 9 left the experiment
